In [1]:
%run ../../bin/imports.ipynb
import string
import matplotlib.image as mpimg
import shutil
from matplotlib import gridspec

## Analyzing TMT results

Import data and get HLA types

In [2]:
tissue_msb_dict = {'DNA_1070865FA1-5013': 'MSB8405PAIR2MS2TMT.raw', 
                   'DNA_1183384FA1-5013': 'MSB46732n46733A.raw', 
                   'DNA_1134036FA1-5013': 'MSB46734n46735A.raw',
                   'DNA_124768F-5013': 'MSB46736n46737A.raw', 
                   'DNA_30686F-5013': 'MSB46738n46739A.raw',
                   'DNA_H2009': 'MSB-9251_PAIR1.raw',
                  'DNA_54147': 'MSB-9251_PAIR2.raw'}

In [3]:
tissue_order = list(tissue_msb_dict.keys())
tissue = tissue_order[0]

Separate raw data

In [4]:
raw_data = pd.read_csv('/rnd/users/rpyke/data/00-DASH/manuscript_data_v2/validation.immunopeptidomics_raw_data_1.csv')


In [5]:
raw_data2 = pd.read_csv('/rnd/users/rpyke/data/00-DASH/manuscript_data_v2/validation.immunopeptidomics_raw_data_2.csv')


In [6]:
raw_data['Source File'].value_counts()

MSB8405PAIR2MS2TMT.raw    8999
MSB8405PAIR1MS2TMT.raw    8618
MSB46736n46737A.raw       6489
MSB46732n46733A.raw       5095
MSB46738n46739A.raw       4656
MSB46734n46735A.raw       3390
Name: Source File, dtype: int64

In [7]:
raw_data2['Source File'].value_counts()

MSB-9251_PAIR1.raw    6050
MSB-9251_PAIR2.raw    1338
Name: Source File, dtype: int64

In [8]:
tissue_msb_dict[tissue]

'MSB8405PAIR2MS2TMT.raw'

In [9]:
for tissue in tissue_order:
    tmp_df = raw_data[raw_data['Source File'] == tissue_msb_dict[tissue]]
    if len(tmp_df) == 0:
        tmp_df = raw_data2[raw_data2['Source File'] == tissue_msb_dict[tissue]]
    print(tissue, len(tmp_df))
    tmp_df.to_csv('/rnd/users/rpyke/data/00-DASH/validation/TMT/peaks/raw/{0}.csv'.format(tissue),
                                                                        index=None)

DNA_1070865FA1-5013 8999
DNA_1183384FA1-5013 5095
DNA_1134036FA1-5013 3390
DNA_124768F-5013 6489
DNA_30686F-5013 4656
DNA_H2009 6050
DNA_54147 1338


Process raw data (this is actually un-necessary this time because Datta pre-processed it for me)

In [10]:
# maybe easier to do it from a script made on vivaldi??? 
def create_cluster_script():

    new_script_file = '/rnd/users/rpyke/code/00-DASH/scripts/process_TMT.sh'
    
    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -cwd\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -pe shm 1\n")
        out_file.write("#$ -l h_vmem=5G\n")
        out_file.write("#$ -N pep_filter\n")
        out_file.write("#$ -q all.q\n")
        out_file.write("#$ -o /rnd/users/rpyke/data/01-netMHCAACR2019/sge_system_files\n")
        out_file.write("#$ -e /rnd/users/rpyke/data/01-netMHCAACR2019/sge_system_files\n")
        out_file.write("\n")


        out_file.write("date\n")
        out_file.write("hostname\n")
        out_file.write("\n")
        
        out_file.write("python /rnd/users/mp-wsrv-hmc01/sherpa/sherpa/filter_tmt_peps.py " + \
                       "--in_file /rnd/users/rpyke/data/00-DASH/validation/TMT/peaks/raw/DNA_1180157FA1-5013.csv " + \
                       "--out_file /rnd/users/rpyke/data/00-DASH/validation/TMT/peaks/raw/DNA_1180157FA1-5013.out.csv " + \
                       "--intensity_colnums 22,16\n")
        
        out_file.write("python /rnd/users/mp-wsrv-hmc01/sherpa/sherpa/filter_tmt_peps.py " + \
                       "--in_file /rnd/users/rpyke/data/00-DASH/validation/TMT/peaks/raw/DNA_1070865FA1-5013.csv " + \
                       "--out_file /rnd/users/rpyke/data/00-DASH/validation/TMT/peaks/raw/DNA_1070865FA1-5013.out.csv " + \
                       "--intensity_colnums 23,17\n")
        
        out_file.write("python /rnd/users/mp-wsrv-hmc01/sherpa/sherpa/filter_tmt_peps.py " + \
                       "--in_file /rnd/users/rpyke/data/00-DASH/validation/TMT/peaks/raw/DNA_1183384FA1-5013.csv " + \
                       "--out_file /rnd/users/rpyke/data/00-DASH/validation/TMT/peaks/raw/DNA_1183384FA1-5013.out.csv " + \
                       "--intensity_colnums 12,18\n")
        
        out_file.write("python /rnd/users/mp-wsrv-hmc01/sherpa/sherpa/filter_tmt_peps.py " + \
                       "--in_file /rnd/users/rpyke/data/00-DASH/validation/TMT/peaks/raw/DNA_1134036FA1-5013.csv " + \
                       "--out_file /rnd/users/rpyke/data/00-DASH/validation/TMT/peaks/raw/DNA_1134036FA1-5013.out.csv " + \
                       "--intensity_colnums 13,19\n")
        
        out_file.write("python /rnd/users/mp-wsrv-hmc01/sherpa/sherpa/filter_tmt_peps.py " + \
                       "--in_file /rnd/users/rpyke/data/00-DASH/validation/TMT/peaks/raw/DNA_124768F-5013.csv " + \
                       "--out_file /rnd/users/rpyke/data/00-DASH/validation/TMT/peaks/raw/DNA_124768F-5013.out.csv " + \
                       "--intensity_colnums 14,20\n")
        
        
        out_file.write("python /rnd/users/mp-wsrv-hmc01/sherpa/sherpa/filter_tmt_peps.py " + \
                       "--in_file /rnd/users/rpyke/data/00-DASH/validation/TMT/peaks/raw/DNA_30686F-5013.csv " + \
                       "--out_file /rnd/users/rpyke/data/00-DASH/validation/TMT/peaks/raw/DNA_30686F-5013.out.csv " + \
                       "--intensity_colnums 15,21\n")
        
        out_file.write("python /rnd/users/mp-wsrv-hmc01/sherpa/sherpa/filter_tmt_peps.py " + \
                       "--in_file /rnd/users/rpyke/data/00-DASH/validation/TMT/peaks/raw/DNA_H2009.csv " + \
                       "--out_file /rnd/users/rpyke/data/00-DASH/validation/TMT/peaks/raw/DNA_H2009.out.csv " + \
                       "--intensity_colnums 12,13\n")
        
        out_file.write("python /rnd/users/mp-wsrv-hmc01/sherpa/sherpa/filter_tmt_peps.py " + \
                       "--in_file /rnd/users/rpyke/data/00-DASH/validation/TMT/peaks/raw/DNA_54147.csv " + \
                       "--out_file /rnd/users/rpyke/data/00-DASH/validation/TMT/peaks/raw/DNA_54147.out.csv " + \
                       "--intensity_colnums 15,14\n")
        
        out_file.write("\n")
        out_file.write("date\n")

In [11]:
create_cluster_script()

Interpreting print statements from output to understand how many peptides are excluded based on polymers

In [12]:
output_df = pd.DataFrame({'Patient': ['DNA_1070865FA1-5013', 'DNA_1183384FA1-5013',
                                      'DNA_1134036FA1-5013', 'DNA_124768F-5013', 'DNA_30686F-5013',
                                      'DNA_H2009', 'DNA_54147'],
                         'Total': [8999, 5095, 3390, 6489, 4656, 6050, 1338],
                         'Long_repeats': [77, 23, 29, 43, 36, 70, 27]})

In [13]:
output_df['Fraction_long_repeats'] = output_df['Long_repeats'] / output_df['Total']

In [14]:
output_df

,Patient,Total,Long_repeats,Fraction_long_repeats
0,DNA_1070865FA1-5013,8999,77,0.008557
1,DNA_1183384FA1-5013,5095,23,0.004514
2,DNA_1134036FA1-5013,3390,29,0.008555
3,DNA_124768F-5013,6489,43,0.006627
4,DNA_30686F-5013,4656,36,0.007732
5,DNA_H2009,6050,70,0.011570
6,DNA_54147,1338,27,0.020179


### Create table with output

In [15]:
def format_hla(x):
    return 'HLA-{0}{1}:{2}'.format(x[0], x.split('*')[1].split(':')[0], x.split('*')[1].split(':')[1])

In [16]:
all_dfs = []
for tissue in tissue_order:
    print(tissue)
    tmp_df = pd.read_csv('/rnd/users/rpyke/data/00-DASH/validation/TMT/peaks/raw/{0}.out.csv'.format(tissue))
    all_dfs.append(tmp_df)


DNA_1070865FA1-5013
DNA_1183384FA1-5013
DNA_1134036FA1-5013
DNA_124768F-5013
DNA_30686F-5013
DNA_H2009
DNA_54147


In [17]:
AAList = list(string.ascii_uppercase)
def get_peptide(peptide):
    pepSeq = [item for item in peptide if item in AAList]
    pepSeq = "".join(pepSeq)
    return pepSeq

unique_peptide_counts = []
all_dfs_filtered = []
for df in all_dfs:
    df['Sequence'] = df.Peptide.apply(get_peptide)
    df = df[df.has_good_signal == 'Y']
    df['Sequence_length'] = df.Sequence.str.len()
    df = df[(df.Sequence_length > 7) & (df.Sequence_length < 12)]
    all_dfs_filtered.append(df)
    unique_peptide_counts.append(len(df.Sequence.unique()))


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Create data table

In [18]:
tissue_msb_dict

{'DNA_1070865FA1-5013': 'MSB8405PAIR2MS2TMT.raw',
 'DNA_1183384FA1-5013': 'MSB46732n46733A.raw',
 'DNA_1134036FA1-5013': 'MSB46734n46735A.raw',
 'DNA_124768F-5013': 'MSB46736n46737A.raw',
 'DNA_30686F-5013': 'MSB46738n46739A.raw',
 'DNA_H2009': 'MSB-9251_PAIR1.raw',
 'DNA_54147': 'MSB-9251_PAIR2.raw'}

In [21]:
sample_letter_map = pd.DataFrame({'Sample': [ 'DNA_1070865FA1-5013',
                                            'DNA_1183384FA1-5013', 'DNA_1134036FA1-5013', 'DNA_124768F-5013',
                                           'DNA_30686F-5013', 'DNA_H2009', 'DNA_54147'],
                                  'Tissue': ['1070865FA1',
                                            '1183384FA1', '1134036FA1', '124768F',
                                           '30686F', 'H2009', '54147'],
                                 'Tissue_letter': ['M', 'N', 'O', 'C', 'P', 'CellLine', 'Q']})

In [22]:
with pd.ExcelWriter('/rnd/users/rpyke/data/00-DASH/tables/Table_S6_immunopeptidomics_peptide_fold_change.xlsx') as writer:  
    for i, tissue in enumerate(tissue_order):
        tissue_name = tissue.split('_')[1].split('-')[0]
        tissue_letter = list(sample_letter_map[sample_letter_map.Tissue == tissue_name].Tissue_letter)[0]
        tissue_dictionary, raw_tissue_dictionary = {}, {}
        df = all_dfs_filtered[i]
        df['fc_log'] = df.fc.apply(np.log2)
        df = df[['Sequence', 'fc_log']]
        df.columns = ['Peptide', 'Log Fold Change']
        df.to_excel(writer, index=None, 
                    sheet_name=tissue_letter)
        